In [1]:
import pandas as pd
df =pd.read_csv('./data/Fake_Real_Data.csv')
df.head()

,Text,label
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake
1,U.S. conservative leader optimistic of common ...,Real
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real
3,Court Forces Ohio To Allow Millions Of Illega...,Fake
4,Democrats say Trump agrees to work on immigrat...,Real


In [2]:
df.label.value_counts()

Fake    5000
Real    4900
Name: label, dtype: int64

In [3]:
df['label_num'] = df.label.map({'Fake':0,'Real':1})
df.head()

,Text,label,label_num
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake,0
1,U.S. conservative leader optimistic of common ...,Real,1
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real,1
3,Court Forces Ohio To Allow Millions Of Illega...,Fake,0
4,Democrats say Trump agrees to work on immigrat...,Real,1


In [4]:
import spacy
nlp = spacy.load('en_core_web_lg')

In [6]:
doc = nlp(df.Text[0])
doc.vector.shape

(300,)

In [7]:
df['Text_vec'] = df.Text.apply(lambda x : nlp(x).vector)

In [8]:
df.head()

,Text,label,label_num,Text_vec
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake,0,"[-0.6759837, 1.4263071, -2.318466, -0.451093, ..."
1,U.S. conservative leader optimistic of common ...,Real,1,"[-1.8355803, 1.3101058, -2.4919677, 1.0268308,..."
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real,1,"[-1.9851209, 0.14389805, -2.4221718, 0.9133005..."
3,Court Forces Ohio To Allow Millions Of Illega...,Fake,0,"[-2.7812982, -0.16120885, -1.609772, 1.3624227..."
4,Democrats say Trump agrees to work on immigrat...,Real,1,"[-2.2010763, 0.9961637, -2.4088492, 1.128273, ..."


In [10]:
df.isna().sum()

Text         0
label        0
label_num    0
Text_vec     0
dtype: int64

In [47]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(df.Text_vec.values, df.label_num, test_size=0.2, stratify=df.label_num, 
                                                 random_state=2023)

In [48]:
x_train

array([array([-2.1908548 ,  0.791702  , -1.4697698 ,  0.1382259 ,  4.1426077 ,
               0.29635197,  0.6382097 ,  4.0531435 , -0.10653446, -1.6306084 ,
               5.42525   ,  1.1756846 , -3.654751  ,  1.144958  ,  0.13231085,
               2.1777413 ,  1.1960903 , -0.69329005, -1.3682019 , -1.6884716 ,
               0.6702646 , -0.9804286 , -1.0975829 ,  0.20606637, -0.20330322,
              -1.4376878 , -2.1110005 , -0.40156287, -0.36346692,  1.0520172 ,
               1.395485  ,  0.3450101 , -1.1077704 , -2.0626895 , -2.2227094 ,
              -0.7660622 , -0.8246554 ,  0.67344886,  1.315362  ,  0.54099536,
               0.2314701 ,  0.5207544 , -0.02104906,  0.7455942 , -2.0717237 ,
               1.4691074 ,  1.1080498 , -2.1067145 , -0.9978273 ,  1.738351  ,
              -1.2710533 ,  2.0844257 ,  0.1578646 , -5.326206  , -0.60088104,
               0.81207466, -0.28130454,  1.1620935 ,  0.83643115, -0.131438  ,
               0.6211622 , -0.8545687 , -0.12173069,

In [49]:
import numpy as np 
x_train_2d = np.stack(x_train)
x_test_2d = np.stack(x_test)

In [50]:
# x_train is not 2d array
x_train_2d

array([[-2.1908548 ,  0.791702  , -1.4697698 , ..., -1.6585066 ,
        -2.032391  ,  1.42908   ],
       [-1.5435404 ,  0.769298  , -1.8126677 , ..., -0.67195827,
        -1.540145  ,  0.38910717],
       [-1.533173  ,  1.0636886 , -2.058168  , ..., -0.7168052 ,
        -2.8014107 ,  1.0071656 ],
       ...,
       [-1.722747  ,  0.89875674, -1.1274978 , ..., -1.0450921 ,
        -2.1223056 ,  1.2484393 ],
       [-1.6660326 ,  0.36219943, -2.2478688 , ..., -0.6778854 ,
        -2.6150985 ,  1.0045427 ],
       [-1.5841519 ,  0.8324813 , -2.0148711 , ..., -0.7741788 ,
        -2.5402944 ,  1.2127181 ]], dtype=float32)

In [51]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(x_train_2d,y_train)

ValueError: Negative values in data passed to MultinomialNB (input X)

In [63]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

clf = Pipeline([
    ('scaler', MinMaxScaler()),
    ('model', MultinomialNB())
])

clf.fit(x_train_2d,y_train)

Pipeline(steps=[('scaler', MinMaxScaler()), ('model', MultinomialNB())])

In [64]:
 clf.score(x_test_2d,y_test)

0.9358585858585858

In [65]:
y_pred = clf.predict(x_test_2d)

In [66]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.95      0.93      0.94      1000
           1       0.93      0.95      0.94       980

    accuracy                           0.94      1980
   macro avg       0.94      0.94      0.94      1980
weighted avg       0.94      0.94      0.94      1980



In [67]:
# Knn is not good with bag of words tdf-Idf because we have 100k values 
# in here we have 300 thats why knn works well here

In [69]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from  sklearn.neighbors import KNeighborsClassifier

clf = Pipeline([
    ('scaler', MinMaxScaler()),
    ('model', KNeighborsClassifier(n_neighbors = 5, metric = 'euclidean'))
])

clf.fit(x_train_2d,y_train)

y_pred = clf.predict(x_test_2d)

print(classification_report(y_test,y_pred))


AttributeError: 'NoneType' object has no attribute 'split'